In [14]:
# Delete all indexes to start fresh.
# curl -X DELETE http://localhost:9200/ubi_queries,ubi_events,
# curl -X DELETE http://localhost:9200/judgments
curl -X DELETE http://localhost:9200/search_quality_eval_query_sets

{"acknowledged":true}


In [ ]:
# Create the UBI indexes
curl -X PUT http://localhost:9200/ubi_queries/_mappings -H "Content-Type: application/json" -d'
{
    "properties": {
      "timestamp": { "type": "date", "format": "strict_date_time" },
      "query_id": { "type": "keyword", "ignore_above": 100 },
      "query": { "type": "text" },
      "query_response_id": { "type": "keyword", "ignore_above": 100 },
      "query_response_hit_ids": { "type": "keyword" },
      "user_query": { "type": "keyword", "ignore_above": 256 },
      "query_attributes": { "type": "flat_object" },
      "client_id": { "type": "keyword", "ignore_above": 100 },
      "application":  { "type":  "keyword", "ignore_above": 100 }
    }
}'

curl -X PUT http://localhost:9200/ubi_events/_mappings -H "Content-Type: application/json" -d'
{
  "properties": {
    "application": { "type": "keyword", "ignore_above": 256 },
    "action_name": { "type": "keyword", "ignore_above": 100 },
    "client_id": { "type": "keyword", "ignore_above": 100 },
    "query_id": { "type": "keyword", "ignore_above": 100 },
    "message": { "type": "keyword", "ignore_above": 1024 },
    "message_type": { "type": "keyword", "ignore_above": 100 },
    "timestamp": {
      "type": "date",
      "format":"strict_date_time",
      "ignore_malformed": true,
      "doc_values": true
    },
    "event_attributes": {
      "dynamic": true,
      "properties": {
        "position": {
          "properties": {
            "ordinal": { "type": "integer" },
            "x": { "type": "integer" },
            "y": { "type": "integer" },
            "page_depth": { "type": "integer" },
            "scroll_depth": { "type": "integer" },
            "trail": { "type": "text",
              "fields": { "keyword": { "type": "keyword", "ignore_above": 256 }
              }
            }
          }
        },
        "object": {
          "properties": {
            "internal_id": { "type": "keyword" },
            "object_id": { "type": "keyword", "ignore_above": 256 },
            "object_id_field": { "type": "keyword", "ignore_above": 100 },
            "name": { "type": "keyword", "ignore_above": 256 },
            "description": { "type": "text",
              "fields": { "keyword": { "type": "keyword", "ignore_above": 256 } }
            },
            "object_detail": { "type": "object" }
          }
        }
      }
    }
  }
}'

In [ ]:
# Index the ESCI data.
cd ./../data/esci && ./index.sh 1> /dev/null 2> /dev/null

In [1]:
# See the query and events count from the indexed ESCI data.
curl -s http://localhost:9200/ubi_queries/_count
curl -s http://localhost:9200/ubi_events/_count

{"count":100000,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0}}
{"count":528374,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0}}


In [3]:
# Create implicit judgments now.
curl -s -X POST "http://localhost:9200/_plugins/search_quality_eval/judgments?click_model=coec&max_rank=20"

# Schedule implicit judgments creation.
# curl -s -X POST "http://localhost:9200/_plugins/search_quality_eval/schedule?id=1&click_model=coec&max_rank=20&job_name=test2&interval=10" | jq
# curl -s "http://localhost:9200/search_quality_eval_scheduler/_search" | jq

{"message": "Implicit judgment generation initiated."}


In [20]:
# See the judgments.
curl -s http://localhost:9200/judgments/_search | jq

{
  "took": 3,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 5000,
      "relation": "eq"
    },
    "max_score": 1,
    "hits": [
      {
        "_index": "judgments",
        "_id": "6ae380c3-4313-4b43-b145-31fbd27a115c",
        "_score": 1,
        "_source": {
          "query_id": "1",
          "judgment": 20.949720670391063,
          "query": "k cups",
          "document": "B07P23H371"
        }
      },
      {
        "_index": "judgments",
        "_id": "400c3242-a4f5-4897-85a2-6db1d7aaf0b4",
        "_score": 1,
        "_source": {
          "query_id": "1",
          "judgment": 22.609514837494114,
          "query": "k cups",
          "document": "B00K2RY8GI"
        }
      },
      {
        "_index": "judgments",
        "_id": "8151d6df-0ab6-428a-ba83-450df26987df",
        "_score": 1,
        "_source": {
          "query_id": "1",
          "judgment":

In [21]:
# Create a query set using PPTSS sampling from the UBI queries.
curl -s -X POST "http://localhost:9200/_plugins/search_quality_eval/queryset?name=test&description=fake&sampling=pptss&query_set_size=50"

{"query_set": "d4857274-3216-4add-afeb-e8c30562270e"}


In [22]:
# Look at the query sets.
curl -s http://localhost:9200/search_quality_eval_query_sets/_search | jq

{
  "took": 1,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 4,
      "relation": "eq"
    },
    "max_score": 1,
    "hits": [
      {
        "_index": "search_quality_eval_query_sets",
        "_id": "46c14597-b4d2-4872-af9c-c5b4ffdef876",
        "_score": 1,
        "_source": {
          "sampling": "pptss",
          "name": "test",
          "description": "fake",
          "created_at": 1732544453255,
          "queries": [
            "k cups",
            "chips",
            "bicycle",
            "shirts long sleeves",
            "best bra without underwire",
            "lamp base only without shade",
            "fitbit charge 3",
            "laptop",
            "ipod",
            "hot pink nike shoes",
            "instant pot",
            "halloween costumes for women",
            "hydro flask",
            "face mask",
            "barbie dolls",
        

In [ ]:
# Run a query set.
curl -s http://localhost:9200/search_quality_eval_run?id=${QUERY_SET_ID}